In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, max_len=140, title="Retriever Results"):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        table.add_row(str(i), src, page, (text[:max_len] + ("…" if len(text) > max_len else "")))

    console.print(table)

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

c:\PythonProject\potenup_10\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 벡터 저장소 만들기
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
db_path = "../7_vectorstore/samsung_2025_db"
collection_name = "samsung2025"

In [5]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf"

print("문서로드중")
loader = PyPDFLoader(file_path)
docs = loader.load()
docs[:5]

문서로드중


[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 0, 'page_label': '1'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 1, 'page_label': '2'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunk = splitter.split_documents(docs)
len(chunk)

237

In [7]:
chunk[5:7]

[Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'total_pages': 87, 'page': 3, 'page_label': '4'}, page_content="사회공헌 분야에서는 2024년 '삼성 청년SW·AI아카데미'에 375억 원을 \n지원해 2,200 명의 청년 소 프트웨어 인재를 양성했으며, 2025년에는 \n마이스터고 졸업생에게까지 교육기회를 확대할 예정입니다. \n또한 자립 준비 청년들의 주 거안정과 미래준비를 지원하는 '삼성 \n희망디딤돌' 활동은 2024년에 충북과 대전센터를 신규로 개소하여 총 \n14,362명을 지원하였으며, 2025년에는 인천센터를 추가로 설립하여 더 \n많은 청년들을 지원할 예정입니다.\n지난해에 이 어 올해 지속가능경영보고서도 글로벌 공시 규 제 \n프레임워크에 맞춰 발 행됐 습니다. 2024년 한 해 동안 회사의  \n지속가능경영 관리체계, 전략, 이 행 활동, 성과 등을 충실 히 담았으며, \n지속가능경영 웹사이트와도 더욱 긴밀히 연계되도록 구성했습니다.\n삼성전자는 불확실성의 시대에 재도 약의 기반을 다지기 위해 지속 \n노력하고 있습니다. '인재와 기술을 바탕으로 최고의 제품과 서비스를 \n창출하여 인류사회에 공헌한다'는 경영철학에 입각해 기술 리더십으로 \n재도약의 기반을 다지고 새로운 영 역에서 미래 성장동력을 확보해 \n나가겠습니다.\n삼성전자는 이해관계자 여러분의 소중한 의견에 늘 귀를 기울이고 있으며, \n앞으로도 지속가능한 성장 기반을 마련하는 

In [8]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
persist_directory = "../7_vectorstore/samsung_2025_db"
collection_name = "samsung2025"

In [9]:
# vectorstore = Chroma.from_documents(
#     documents = chunk,
#     collection_name = collection_name,
#     persist_directory = persist_directory,
#     embedding=embedding
# )
load_vectorstore = Chroma(
    persist_directory=persist_directory,
    collection_name=collection_name,
    embedding_function=embedding
)

# 1. 키워드 + 기본 탐색기 = 하이브리드(ensamble)

In [10]:
ret_similarity = load_vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={
        "k":5
    })

In [11]:
db_docs = load_vectorstore.get(include = ["documents", "metadatas"])

In [12]:
db_docs['documents'][:5]

['삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future',
 '삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\n삼성전자 지속가능경영보고서 2025\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n준법과 윤리경영\n[DX부문] \n추진체계와 주요성과 \n기후변화 \n자원순환\n수자원\n오염물질\n경제성과\n사회성과\n환경성과 \n사업부문별 환경성과 \n[DS부문] \n추진체계와 주요성과 \n기후변화 \n자원순환\n수자원\n오염물질\n독립된 인증인의 인증보고서\nScope 1, 2 온실가스 배출량 검증 의견서 \nScope 3 온실가스 배출량 검증 의견서 \nGRI Index\nTCFD 대조표\nSASB 대조표\nAbout This Report \n임직원\n공급망\n사회공헌\n개인정보보호와 보안\n제품 품질과 안전\nOur Company\nPrinciple\nPlanet\nFacts & Figures Appendix \nPeople\n04\n05\n06\n07\n09\n59\n11\n12\n16\n18\n20\n62\n63\n68\n72\n21\n22\n27\n29\n32\n76\n77\n78\n80\n82\n84\n86\n35\n45\n51\n53\n55',
 '삼성전자 지속가능경영보고서 2025\n03\nOur Company AppendixFacts & Figures PrinciplePlanet People\nOur Company\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n

In [13]:
db_docs['metadatas'][:5]

[{'trapped': '/False',
  'producer': 'Adobe PDF Library 15.0',
  'page': 0,
  'moddate': '2025-09-04T16:51:11+09:00',
  'total_pages': 87,
  'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'creationdate': '2025-07-10T16:11:16+09:00',
  'page_label': '1'},
 {'moddate': '2025-09-04T16:51:11+09:00',
  'trapped': '/False',
  'producer': 'Adobe PDF Library 15.0',
  'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'page_label': '2',
  'page': 1,
  'total_pages': 87,
  'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'creationdate': '2025-07-10T16:11:16+09:00'},
 {'creator': 'Adobe InDesign 15.1 (Macintosh)',
  'trapped': '/False',
  'creationdate': '2025-07-10T16:11:16+09:00',
  'page': 2,
  'page_label': '3',
  'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
  'producer': 'Adobe PDF Library 15.0',
  'total_pages': 87,
  'moddate': '2025-09-04T16:51:11+09:00'}

In [14]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

In [15]:

from langchain_core.documents import Document

bm_doc = [] # 사실은 chunk를 쓰면 되지만
# 우리는 vectorstore 를 처음에 만들고 더이상 똑같은걸 다시 만들지 않을 것이므로 
# load_vectorstore._collection 으로 가져와서 chunk 같은 Document 리스트를 만들어야 함
#  
for content, meta in zip(db_docs["documents"],db_docs["metadatas"]):
    bm_doc.append(Document(page_content=content, metadata=meta))

bm_doc[:3]

[Document(metadata={'trapped': '/False', 'producer': 'Adobe PDF Library 15.0', 'page': 0, 'moddate': '2025-09-04T16:51:11+09:00', 'total_pages': 87, 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'page_label': '1'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'page_label': '2', 'page': 1, 'total_pages': 87, 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'creationdate': '2025-07-10T16:11:16+09:00'}, page_content='삼성전자 지속가능경영보고서 2025 02AppendixFacts & Figures PrinciplePlanet PeopleOur Company삼성전자 지속가능경영보고서 2025 02\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards \n a Sustainable Future\

In [16]:
bm25 = BM25Retriever.from_documents(bm_doc) # 키워드 기반 검색기 
bm25.k = 5 

In [17]:
# 벡터 검색기 + bm25 = 하이브리드 검색기 완성
ret_hybrid = EnsembleRetriever(
    retrievers = [ret_similarity, bm25],
    weights = [0.5,0.5]
)

In [18]:
question = "삼성 전자의 2025년 전망은?"
result = ret_hybrid.invoke(question)

In [19]:
result

[Document(id='3871f7b3-d7ad-4e72-9dd3-4c99be99840c', metadata={'creationdate': '2025-07-10T16:11:16+09:00', 'producer': 'Adobe PDF Library 15.0', 'total_pages': 87, 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page_label': '1', 'trapped': '/False', 'moddate': '2025-09-04T16:51:11+09:00', 'page': 0}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'),
 Document(metadata={'page': 51, 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'creationdate': '2025-07-10T16:11:16+09:00', 'producer': 'Adobe PDF Library 15.0', 'page_label': '52', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'trapped': '/False', 'moddate': '2025-09-04T16:51:11+09:00', 'total_pages': 87}, page_content='삼성전자 지속가능경영보고서 2025\n52\n삼성 드림클래스\n삼성 드림클래스는 임직원의 후원과 참여를 바탕으로 교육 여건이 부족한 \n국내 중학생들이 꿈을 찾고 미래를 준비할 수 있도록 멘토링과 맞춤형 \n진로 교육을 제공하는 프 로그램으로 대학생 멘 토, 

In [20]:
rich_docs(result, title="삼성 전자 전망")

                                                  삼성 전자 전망                                                   
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey       │
│    │                                                  │      │ Towards   a Sustainable Future A Journey         │
│    │                                                  │      │ Towards  a Sustainable Future                    │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   52 │ 삼성전자 지속가능경영보고서 2025 52 삼성         │
│    │                                                  │      │ 드림클래스 삼성 드림클래스는 임직원의 후원과     │
│    │                                                  │      │ 참여를 바탕으로 교육 여건이 부족한  국내         │
│    │                                                  │      │ 중학생들이 꿈을 찾고 미래를 준비할 수 있도록     │
│    │                                                  │      │ 멘토링과 맞춤형  진로 교육을 제공하는 프         │
│    │                                                  │      │ 로그램으로 대학생 멘 토, …                       │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   63 │ 삼성전자 지속가능경영보고서 2025 63 사회성과     │
│    │                                                  │      │ 준법·윤리경영 2022년 2023년 2024년 [컴플라이언 … │
│    │                                                  │      │ 교육] 컴플라이언스 교육 1) 명 126,867  138,742   │
│    │                                                  │      │ 138,414 [부정 예방 교육] 부정 예방 교육 2) 명    │
│    │                                                  │      │ 254,045  …                                       │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   76 │ 삼성전자 지속가능경영보고서 2025 76 독립된       │
│    │                                                  │      │ 인증인의 인증보고서 Our Company AppendixFacts &  │
│    │                                                  │      │ Figures PrinciplePlanet People                   │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   51 │ 프로그램을 운영하고 있습니다. 추진 방향          │
│    │                                                  │      │ 삼성전자는 ‘함께가요 미래로! Enabling P eople’   │
│    │                                                  │      │ 비전 아래 , 교육의  기회에서 소외되는 학생 없이  │
│    │                                                  │      │ 모든 청소년에게 양질의 교육을 제공하는           │
│    │                                                  │      │ 프로그램을 운영하며, 삼성전자의 경영 노하우를    │
│    │                                                  │      │ 중소기업…                                        │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   77 │ 삼성전자 지속가능경영보고서 2025 77 Scope 1, 2   │
│    │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company          │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│  8 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 필수적으로 관리해야할 대상으로 인식합니다.       │
│    │                                                  │      │ 삼성전자 반도체의 주요  기술은 산업기술보호법 1… │
│    │                                                  │      │ 과 첨단전략산업법 2) 에 따라 보호되며, 매년 정 … │
│    │ 

### 2. 압축 검색기 (compression retreiver) 
- 검색된 문서가 길 때 -> llm 을 이용해서 내용을 압축해보기 
- 문서 내용이 너무 파편화 되어 있는 경우 -> 압축 진행 -> 찌꺼기가 제거
- 비용문제 -> 각 문서별로 전부 압축을 진행해서 사용하기 때문

In [21]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [22]:
compresser = LLMChainExtractor.from_llm(model)
com_retriever = ContextualCompressionRetriever(
    base_compressor=compresser,
    base_retriever=ret_hybrid
)

In [23]:
question = "삼성 전자의 목표와 기준 년도만 간단히 알려줘"
com_result = com_retriever.invoke(question)
com_result

[Document(metadata={'creator': 'Adobe InDesign 15.1 (Macintosh)', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page': 3, 'moddate': '2025-09-04T16:51:11+09:00', 'producer': 'Adobe PDF Library 15.0', 'creationdate': '2025-07-10T16:11:16+09:00', 'trapped': '/False', 'page_label': '4', 'total_pages': 87}, page_content="삼성전자는 2022년 9월 발표한 '新환경경영전략'을 기반으로 탄소중립 달성, 자원순환 극대화, 그리고 기술 혁신을 통한 환경 난제 해결을 위해 노력하고 있습니다.  \nDX(Device eXperience)부문은 2030년 탄소중립 달성을 목표로  \nDS(Device Solutions)부문은 2050년 탄소중립 달성을 목표로"),
 Document(metadata={'page_label': '83', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'moddate': '2025-09-04T16:51:11+09:00', 'total_pages': 87, 'page': 82, 'creationdate': '2025-07-10T16:11:16+09:00', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'trapped': '/False', 'producer': 'Adobe PDF Library 15.0'}, page_content='c)  기후변화 관련 리스크와 기회 관리를 \n위한 기업의 목표와 목표 대비 성과\n삼성전자는 기후변화 관련 리스크와 기회 관리를 위한 기업의 목표와 목표 대비 성과를 지속가능경영보고서, CDP 등

In [24]:
for doc in com_result:
    print(doc.page_content, "\n", "-"*60)

삼성전자는 2022년 9월 발표한 '新환경경영전략'을 기반으로 탄소중립 달성, 자원순환 극대화, 그리고 기술 혁신을 통한 환경 난제 해결을 위해 노력하고 있습니다.  
DX(Device eXperience)부문은 2030년 탄소중립 달성을 목표로  
DS(Device Solutions)부문은 2050년 탄소중립 달성을 목표로 
 ------------------------------------------------------------
c)  기후변화 관련 리스크와 기회 관리를 
위한 기업의 목표와 목표 대비 성과
삼성전자는 기후변화 관련 리스크와 기회 관리를 위한 기업의 목표와 목표 대비 성과를 지속가능경영보고서, CDP 등을 통해 공개하고 있습니다. P.11, 22, 
 ------------------------------------------------------------
삼성전자 지속가능경영보고서 2025 
 ------------------------------------------------------------
삼성전자 지속가능경영보고서 2025  
삼성전자주식회사는 경제·사회·환경적 가치 창출 성과를 다양한 이해관계자와 투명하게 소통하기 위해 2025년 열여덟 번째 지속가능경영보고서를 발간합니다.  
보고 기간  
2024년 1월1일부터 2024년 12월 31일까지의 경제·사회·환경적 성과와 활동을 담고 있으며, 일부 성과에 대해서는 2025년 6월 까지의 정보를 포함하고 있습니다. 
 ------------------------------------------------------------
삼성전자 지속가능경영보고서 2025 
 ------------------------------------------------------------
삼성전자는 ‘함께가요 미래로! Enabling P eople’ 비전 아래 , 교육의 
기회에서 소외되는 학생 없이 모든 청소년에게 양질의 교육을 제공하는 
프로그램을 운영하며, 삼성전자의 경영 노하우를 중소

In [25]:
rich_docs(com_result, title="삼성 전자 목표와 기준 년도")

                                            삼성 전자 목표와 기준 년도                                             
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자는 2022년 9월 발표한 '新환경경영전략'을   │
│   │                                                  │      │ 기반으로 탄소중립 달성, 자원순환 극대화, 그리고   │
│   │                                                  │      │ 기술 혁신을 통한 환경 난제 해결을 위해 노력하고   │
│   │                                                  │      │ 있습니다.   DX(Device eXperience)부문은 2030년    │
│   │                                                  │      │ 탄소중립 달성을 목표로   …                        │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │   83 │ c)  기후변화 관련 리스크와 기회 관리를  위한      │
│   │                                                  │      │ 기업의 목표와 목표 대비 성과 삼성전자는 기후변화  │
│   │                                                  │      │ 관련 리스크와 기회 관리를 위한 기업의 목표와 목 … │
│   │                                                  │      │ 대비 성과를 지속가능경영보고서, CDP 등을 통해     │
│   │                                                  │      │ 공개하고 있습니다. P.11, 22,                      │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025                  │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025                  │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다.   보고 기간   2024년 1월1일부터       │
│   │                                                  │      │ 2024년 12월 …                                     │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   52 │ 삼성전자 지속가능경영보고서 2025                  │
│ 6 │ Samsung_Electronics_Sustainability_Report_2025_… │   51 │ 삼성전자는 ‘함께가요 미래로! Enabling P eople’    │
│   │                                                  │      │ 비전 아래 , 교육의  기회에서 소외되는 학생 없이   │
│   │                                                  │      │ 모든 청소년에게 양질의 교육을 제공하는            │
│   │                                                  │      │ 프로그램을 운영하며, 삼성전자의 경영 노하우를     │
│   │                                                  │      │ 중소기업과 스타트업에  전수합니다.   삼성전자는…  │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

# 2-2. 임베딩 기반 경량 압축(비용 X)

In [26]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

emb_filter = EmbeddingsFilter(
    embeddings=embedding,
    similarity_threshold=0.2
)
comp_embed = ContextualCompressionRetriever(
    base_compressor=emb_filter,
    base_retriever=ret_similarity
)
comp_embed_result = comp_embed.invoke(question)

In [27]:
rich_docs(comp_embed_result, title="임베딩 기반 필터기법")

                                               임베딩 기반 필터기법                                                
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   84 │ 삼성전자 지속가능경영보고서 2025 84 코드 공시     │
│   │                                                  │      │ 항목 참고 페이지 및 답변 제품 안전 TC-HW-230a.1   │
│   │                                                  │      │ 제품의 정보보안 관련 리스크를 발견하고 이를       │
│   │                                                  │      │ 개선하는 방법 설명 삼성전자는 개인정보보호팀장,   │
│   │                                                  │      │ 정보보호센터장이 개인정보보호 운영위원회와 보안…  │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   78 │ 삼성전자 지속가능경영보고서 2025 78 Scope 3       │
│   │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company           │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

# 3. 리랭커

In [28]:
ret_similarity = load_vectorstore.as_retriever(
    search_type="similarity", 
    search_kwargs={
        "k":10
    })
question = "삼성과 현대 중에서 어느 기업이 더 나아?"
ret_similarity_result = ret_similarity.invoke(question)
rich_docs(ret_similarity_result, title="기업 비교")

                                                     기업 비교                                                     
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ Our Company CEO 메시지 회사소개 기업 지배구조    │
│    │                                                  │      │ 중대성 평가 이해관계자 소통 04 05 06 …           │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                                  │      │ 글로벌 제조&인프라  총괄장인 CSO(Chief Safety    │
│    │                                                  │      │ Officer)를 중심으로 2030년까지 상주  협력회사    │
│    │                                                  │      │ 안전보건 역량과 관리체계 …                       │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke AI   │
│    │                                                  │      │ 로봇청소기 보안 인증 획득 2024년에는 Bespoke AI  │
│    │                                                  │      │ 스팀 로봇청소기가 KISA 개인정보보호중심설계      │
│    │                                                  │      │ (PbD, Privacy by Design) 인증과 KISA IoT 보안    │
│    │                                                  │      │ 인증 중 …                                        │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경영 관리 수준 향상을 위한 실사와 개     │
│    │                                                  │      │ 선조치 이행, 정보공개 등의 성과는 해 당 부서의   │
│    │                                                  │      │ 경영진과 실무자들의 KPI로 반영되며 목표 달성은   │
│    │                            

In [29]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker


model_name = "cross-encoder/ms-marco-MiniLM-L6-v2"
max_length = 512
hf_ce = HuggingFaceCrossEncoder(
    model_name=model_name, model_kwargs={"device": "cuda", "max_length": max_length}
)

In [30]:
compressor = CrossEncoderReranker(model=hf_ce, top_n=10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=ret_similarity
)

compressed_result = compression_retriever.invoke(question)
rich_docs(compressed_result, title="리랭크 결과")

                                                    리랭크 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동규범 삼성전자는 제품과 서비스를 제공하는     │
│    │                                                  │      │ 모든 협력회사에 노동인권, 환경, 안전보건, 윤리   │
│    │                                                  │      │ 분야를 포함하는 현지 법규와    RBA(Responsible   │
│    │                                                  │      │ Business Alliance) 1)  행…                       │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경영 관리 수준 향상을 위한 실사와 개     │
│    │                                                  │      │ 선조치 이행, 정보공개 등의 성과는 해 당 부서의   │
│    │                                                  │      │ 경영진과 실무자들의 KPI로 반영되며 목표 달성은   │
│    │                                                  │      │ 급여, 인센티브 등 보상과 연계됩니다. 추진 방향   │
│    │                                                  │      │ 삼성전자…                                        │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke AI   │
│    │                                                  │      │ 로봇청소기 보안 인증 획득 2024년에는 Bespoke AI  │
│    │                                                  │      │ 스팀 로봇청소기가 KISA 개인정보보호중심설계      │
│    │                                                  │      │ (PbD, Privacy by Design) 인증과 KISA IoT 보안    │
│    │                                                  │      │ 인증 중 …                                        │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                              

In [33]:
print(f"**쿼리: {question}**\n")
print(f"--- 재순위화된 상위 {compressor.top_n}개 문서 ---\n")

for i, doc in enumerate(compressed_result):
    # Cross-Encoder의 점수는 'relevance_score' 메타데이터에 저장됨
    score = doc    
    print(f"[{i+1}] 점수: {score}")
    print(f"내용: {doc.page_content[:150]}...\n")

**쿼리: 삼성과 현대 중에서 어느 기업이 더 나아?**

--- 재순위화된 상위 10개 문서 ---

[1] 점수: page_content='삼성전자 지속가능경영보고서 2025
45
협력회사 행동규범
삼성전자는 제품과 서비스를 제공하는 모든 협력회사에 노동인권, 환경, 안전보건, 윤리 분야를 포함하는 현지 법규와   
RBA(Responsible Business Alliance)
1)
 행동규범을 반영해 제정한 삼성전자 협력회사 행동규범 을 준수할 것을 
요구하며, 협력회사가 자율적으로 행동규범을 준수하고 지속가능경영을 실천할 수 있도록 행동규범 가이드 를  
제공합니다.
2024년에는 업데이트 된 RBA 행동규범의 내용을 반영하여 협력회사 행동규범을 개정하였으며, 1차 협력회사가 
2차 협력회사를 대상으로 실사와 개선관리를 이행하도록 하는 '공급망 참여와 책임 이행 조항'을 신설하였습니다.
1) 글로벌 공급망에서 사회적 책임을 전담하는 산업 연합체인 책임있는 비즈니스 연합
추진 체계
삼성전자는 이사회 산하 지속가능경영위원회와 DX부문의 지속가능경영협의회, DS부문의 ESG경영협의회를 
중심으로 공급망 내 안건들을 관리·감독하며, 협력회사 노동인권 관련 이슈는 노동인권협의회에서 함께 다루고 
있습니다. 2024년 지속가능경영위원회는 EU 공급망 실사 지침 대응방안을 논의하였습니다.
지속가능경영추진센터는 EU CSDDD(Corporate Sustainability Due Diligence Directive) 등 공급망 규제 
대응의 컨트롤타워 역할을 수행하며, 공급망 관련 정책의 실행은 상생협력센터, 수원지원센터, 생산기술연구소와 
각 사업장 내 전담 조직이 담당하고 있습니다. 상생협력센터는 협력회사와 상호 신뢰 기반의 상생협력 환경을 
조성하기 위하여 제조 협력회사와 구매 담당자 대상 행동규범을 제정·전파하며, 협력회사 운영·점검과 역량강화 
활동을 지원합니다. 또한 수원지원센터는 삼성전자 사업장 내 비제조 협력회사, 생산기술연구소는 외 주 생산 
협력회사를